<a href="https://colab.research.google.com/github/sanjit1995/MothersDayML_HackerEarth/blob/master/MothersDayML_HE_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import re
import numpy as np

read data and separate

In [0]:
train = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/MothersDayML_HE/train.csv")
test = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/MothersDayML_HE/test.csv")

In [0]:
train['sentiment_class'].value_counts()

 0    1701
-1     769
 1     765
Name: sentiment_class, dtype: int64

In [0]:
pd.crosstab(index = train['sentiment_class'], columns="Total count")

In [0]:
train.isnull().sum()

In [0]:
x_data = train.loc[:, ['original_text']]
y_data = train['sentiment_class']

In [0]:
test_data = test.loc[:, ['original_text']]

In [0]:
x_data.shape

(3235, 1)

In [0]:
y_data.shape

(3235,)

In [0]:
test_data.shape

(1387, 1)

remove web urls from the texts and special characters except # and '

for train

In [0]:
x_data['new_text'] = x_data['original_text'].str.replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True).replace(r'pic.\S+', '', regex=True).replace(r'photos.app\S+', '', regex=True).replace(r'twitter.com\S+', '', regex=True)

In [0]:
x_data['new_text'] = x_data['new_text'].map(lambda x: re.sub(r'[^A-Za-z\s\#\']+', '', x))

for test

In [0]:
test_data['new_text'] = test_data['original_text'].str.replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True).replace(r'pic.\S+', '', regex=True).replace(r'photos.app\S+', '', regex=True).replace(r'twitter.com\S+', '', regex=True)

In [0]:
test_data['new_text'] = test_data['new_text'].map(lambda x: re.sub(r'[^A-Za-z\s\#\']+', '', x))

In [0]:
pd.set_option('display.max_colwidth', None)
x_data.head(3)

In [0]:
test_data.head(3)

,original_text,new_text
0,"3. Yeah, I once cooked potatoes when I was 3 y...",Yeah I once cooked potatoes when I was years...
1,"Happy Mother's Day to all the mums, step-mums,...",Happy Mother's Day to all the mums stepmums gr...
2,"I love the people from the UK, however, when I...",I love the people from the UK however when I a...


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer_test = TfidfVectorizer(
    min_df=1,  # min count for relevant vocabulary
    max_features=12000,  # maximum number of features
    # by their corresponding  ASCII char
    lowercase=True,
    analyzer='word',  # features made of words
    ngram_range=(1, 4),  # features made of a single tokens
    use_idf=True,  # enable inverse-document-frequency reweighting
    smooth_idf=True,  # prevents zero division for unseen words
    sublinear_tf=False)

tfidf_df_test = tfidf_vectorizer_test.fit_transform(test_data['new_text'])

In [0]:
tfidf_vectorizer_test.get_feature_names()

In [50]:
len(list(set(tfidf_vectorizer.get_feature_names()) - set(tfidf_vectorizer_test.get_feature_names())))

6315

fit_transform the train_data

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(
    min_df=1,  # min count for relevant vocabulary
    max_features=6000,  # maximum number of features
    # by their corresponding  ASCII char
    lowercase=True,
    analyzer='word',  # features made of words
    ngram_range=(1, 3),  # features made of a single tokens
    use_idf=True,  # enable inverse-document-frequency reweighting
    smooth_idf=True,  # prevents zero division for unseen words
    sublinear_tf=False)

tfidf_df = tfidf_vectorizer.fit_transform(x_data['new_text'])

just transform the test_data according to train_data

In [0]:
x_test = tfidf_vectorizer.transform(test_data['new_text'])

In [0]:
tfidf_vectorizer.get_feature_names()

In [0]:
tfidf_df

In [0]:
x_train = tfidf_df.copy()
y_train = y_data.copy()

do oversampling

In [116]:
from imblearn.combine import SMOTETomek
smk = SMOTETomek(sampling_strategy='not majority', random_state=42)
X_res,Y_res=smk.fit_sample(x_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [0]:
X_res.shape

(5047, 20000)

In [74]:
unique, counts = np.unique(Y_res, return_counts=True)
dict(zip(unique, counts))

{-1: 1686, 0: 1677, 1: 1690}

In [75]:
Y_res.shape

(5053,)

In [76]:
x_train.shape

(3235, 15000)

In [77]:
y_train.shape

(3235,)

In [78]:
x_test.shape

(1387, 15000)

Training part

using LogisticRegression

In [0]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=300, verbose=2)
model.fit(X_res, Y_res)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=300,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=2,
                   warm_start=False)

using SGDClassifier

In [61]:
from sklearn.linear_model import SGDClassifier
model = SGDClassifier()
model.fit(X_res, Y_res)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [110]:
y_pred = model.predict(x_test)
y_pred

array([ 1,  0,  0, ..., -1, -1, -1])

Using NaiveBayes Classifier

In [117]:
from sklearn.naive_bayes import GaussianNB 
model = GaussianNB() 
model.fit(X_res.toarray(), Y_res) 

GaussianNB(priors=None, var_smoothing=1e-09)

In [118]:
y_pred = model.predict(x_test.toarray())
y_pred

array([-1, -1, -1, ..., -1,  0, -1])

using xgboost

In [109]:
from xgboost.sklearn import XGBClassifier
model = XGBClassifier(max_depth=5, objective='multi:softprob', n_estimators=1000, num_classes=3)

model.fit(X_res, Y_res) 

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=1000, n_jobs=1,
              nthread=None, num_classes=3, objective='multi:softprob',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=None, silent=None, subsample=1, verbosity=1)

In [66]:
len(y_pred)

1387

In [119]:
unique, counts = np.unique(y_pred, return_counts=True)
dict(zip(unique, counts))

{-1: 398, 0: 575, 1: 414}

In [0]:
pred = pd.DataFrame(y_pred)
sub_df = test
sub_df = sub_df.drop(['original_text','lang','retweet_count','original_author'], axis=1)
datasets = pd.concat([sub_df[['id']],pred], axis=1)
datasets.columns = ['id','sentiment_class']
datasets.sentiment_class = datasets.sentiment_class.astype(int)
datasets.to_csv('/content/my_submission75.csv', index=False)

In [0]:
datasets

In [0]:
indices = np.argsort(tfidf_vectorizer.idf_)[::-1]
features = tfidf_vectorizer.get_feature_names()
top_n = 2
top_features = [features[i] for i in indices[:top_n]]
top_features

['naturephotography', 'marina']

In [0]:
feature_array = np.array(tfidf_vectorizer.get_feature_names())
tfidf_sorting = np.argsort(response.toarray()).flatten()[::-1]

n = 3
top_n = feature_array[tfidf_sorting][:n]
top_n